<a href="https://colab.research.google.com/github/0xVolt/cemetery-of-culture/blob/main/year-3/neural-networks/6-sentiment-analysis/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [2]:
# Importing the required libs
import numpy as np  # linear algebra
import pandas as pd  # data processing
import keras
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


only keeping the required coloumns


In [3]:
data = pd.read_csv('Sentiment.csv')
data = data[['text', 'sentiment']]


In [4]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

print(data[data['sentiment'] == 'Positive'].size)
print(data[data['sentiment'] == 'Negative'].size)

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)


4472
16986


the variables embed_dim, lstm_out, batch_size, droupout_x are hyperparameters (the parameters used to control the learning process By contrast the values of the other parameters are derived via traning)thier values are intuitive and can be played with to achive better results


softmax activation funtion is used as the network uses catagorical crossentropy


In [5]:
embed_dim = 128
lstm_out = 196
model = Sequential()  # each layer has one input tensor and one output tensor
model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))

# dropsout 1D feature maps insted of individual elements
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 28, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


traning and testing dataset


In [6]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


traning the model


In [7]:
batch_size = 32
model.fit(X_train, Y_train, epochs=10, batch_size=batch_size, verbose=2)


Epoch 1/10
225/225 - 17s - loss: 0.4417 - accuracy: 0.8154 - 17s/epoch - 75ms/step
Epoch 2/10
225/225 - 14s - loss: 0.3217 - accuracy: 0.8638 - 14s/epoch - 64ms/step
Epoch 3/10
225/225 - 14s - loss: 0.2784 - accuracy: 0.8831 - 14s/epoch - 60ms/step
Epoch 4/10
225/225 - 12s - loss: 0.2502 - accuracy: 0.8972 - 12s/epoch - 52ms/step
Epoch 5/10
225/225 - 11s - loss: 0.2288 - accuracy: 0.9064 - 11s/epoch - 50ms/step
Epoch 6/10
225/225 - 12s - loss: 0.2026 - accuracy: 0.9175 - 12s/epoch - 54ms/step
Epoch 7/10
225/225 - 13s - loss: 0.1785 - accuracy: 0.9290 - 13s/epoch - 57ms/step
Epoch 8/10
225/225 - 14s - loss: 0.1608 - accuracy: 0.9349 - 14s/epoch - 62ms/step
Epoch 9/10
225/225 - 12s - loss: 0.1508 - accuracy: 0.9371 - 12s/epoch - 52ms/step
Epoch 10/10


extracting a validation set and mesuring the score and accuracy


In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))


64/64 - 2s - loss: 0.5063 - accuracy: 0.8236 - 2s/epoch - 24ms/step
score: 0.51
acc: 0.82


In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):

    result = model.predict(X_validate[x].reshape(
        1, X_test.shape[1]), batch_size=1, verbose=2)[0]

    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1

    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")
